# Human Activities Recognition - 2020

### By: Kashif Siddiqui

## Description: 

### Load Model

In [1]:
import os
import random
import numpy as np
import tensorflow as tf
from scipy.io import loadmat

import create_model as cmodel
import tensorflow as tf
from time import time

os.environ["CUDA_VISIBLE_DEVICES"]="1"
tf.random.set_seed(1234)

fs = 20 # sampling frequency

activity_names = {}
activity_names['A'] = 'Walking'
activity_names['B'] = 'Jogging'
activity_names['C'] = 'Stairs'
activity_names['D'] = 'Sitting'
activity_names['E'] = 'Standing'
activity_names['F'] = 'Typing'
activity_names['G'] = 'Brushing Teeth'
activity_names['H'] = 'Eating Soup'
activity_names['I'] = 'Eating Chips'
activity_names['J'] = 'Eating Pasta'
activity_names['K'] = 'Drinking from Cups'
activity_names['L'] = 'Eating Sandwich'
activity_names['M'] = 'Kicking (Soccer Ball)'
activity_names['O'] = 'Playing Catch w/Tennis Ball'
activity_names['P'] = 'Dribbling Basketball'
activity_names['Q'] = 'Writing'
activity_names['R'] = 'Clapping'
activity_names['S'] = 'Folding Clothes'

In [2]:
def read_person_activity(input_directory, file_name):
    full_name = input_directory + '/' + file_name
    activity_dic = {}
    for indx in range(ord('A'),ord('S')+1): # Label for activities are in range 'A' to 'S' (except 'N')
        activity_dic[(chr(indx))] = []
    activity_dic.pop('N') # There is NOT label 'N' in the dataset

    with open(full_name,'r') as f:
        sample = f.readlines()
    sample_len = len(sample)

    for i in range(sample_len): 
        tmp = sample[i].split(',') # e.g. tmp looks like: '1600,A,252207666810782,-0.36476135,8.793503,1.0550842;\n'
        activity_dic[tmp[1]].append([tmp[3], tmp[4], tmp[5].split(';')[0]])
    subject_code = tmp[0]
        
    return subject_code, activity_dic

In [3]:
def full_signal(subj_activities, action, activity_names):
    activity = np.asarray(subj_activities[action],dtype=np.float64)
    #activity = np.asarray(subj_activities[action])
    x = activity[0:,[0]]
    y = activity[0:,[1]]
    z = activity[0:,[2]]
    data = subj_activities[action]
    label2 = activity_names[action]
    #plot3(x,y,z,fs, activity_names[action]+'_0-end seconds')
    return x,y,z, data, label2

In [4]:
for i in [65,66,67,82]:
    print(i)

65
66
67
82


In [5]:
input_directory = '/dataset/HAR/wisdm-dataset/raw/phone/accel'
file_name = 'data_1600_accel_phone.txt'
subj_code, subj_activities = read_person_activity(input_directory, file_name)
x,y,z,data,label = full_signal(subj_activities, chr(65), activity_names)
print(x.shape)

#print(subj_activities)


val = subj_activities.values
temp = list(subj_activities.values())
print(temp[0][0][0])
#arr = np.array([[1, 2, 3, 7], [4, 5, 6, 8]])
#arr= np.array([x,y,z])
#train_data = []
#train_data.append(arr)
#train_data.append(arr)
#train_data = np.asarray(train_data,dtype=np.float64)
#print(train_data)
#print(train_data.shape)
#print(len(train_data[0][0]))

(3574, 1)
-0.36476135


In [6]:
def load_data2D(input_directory):
    '''
    Input Arguements:
    directory = Full path to the folder containing dataset, containing the .mat and .hea files
    Output:
    returns numpy arrays of train_data,train_labels,val_data,val_labels
    The code from Calvin creats the filenames in the order of their name, but here I used listdir for the list of files
    '''
    input_files = []
    for f in os.listdir(input_directory):
        if os.path.isfile(os.path.join(input_directory, f)):
            input_files.append(f)
    #random.shuffle(input_files)
    num_files = len(input_files)
    print('total number of files: ', num_files)
    #classes = get_classes(input_directory,input_files)

    # Create training/val/test split (80/10/10)

    train_split = int(num_files*0.8) 
    val_split = int(num_files*0.1)
    test_split = int(num_files*0.1)
    print('Number of files for trainig: ', train_split)
    print('Number of files for validation: ', val_split)
    print('Number of files for testing: ', test_split)

    # Lists to hold data and labels
    train_data = []
    train_labels = []
    val_data = []
    val_labels = []

    
    
    # Create Training Data and Labels
    for file_number in range(train_split):
        if (file_number+1)%500 == 0:
            print('{} files were loaded for training!'.format(file_number+1))

        # Load file for data
        subject_code, activity_dic = read_person_activity(input_directory, input_files[file_number])

        # This loop read all 12 leads one by one and put them together into "window" list, then adds "window" to the train_data
        for i in [65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83]: #no 78 in the list for "N"
            
            x,y,z, data, label = full_signal(activity_dic, chr(i), activity_names)
            data_app= np.array([x,y,z])
            
            train_data.append(data_app)

            train_labels.append(chr(i))
                
                
    # Repeat the above code for validation data
    for file_number in range(train_split,num_files - test_split):
        if (file_number+1)%500 == 0:
            print('{} files were loaded for validation!'.format(file_number+1-train_split))

        # Load file for data
        subject_code, activity_dic = read_person_activity(input_directory, input_files[file_number])

        # This loop read all 12 leads one by one and put them together into "window" list, then adds "window" to the train_data
        for i in [65,66,67,68,69,70,71,72,73,74,75,76,77,79,80,81,82,83]: #no 78 in the list for "N"
            
            x,y,z, data, label = full_signal(activity_dic, chr(i), activity_names)
            data_app= np.array([x,y,z])
            val_data.append(data_app)

            val_labels.append(chr(i))

    c = 0
    with open('HAR_test_files_00.txt', 'w') as f:
        for file_number in range(train_split+val_split,num_files):
            c += 1
            f.write(input_files[file_number]+ '\n')
    print('number of test signals = ', c) 


    print('train_data type:',  type(train_data))
    print('train_labels  type:',  type(train_labels))
    print('val_data  type:',  type(val_data))
    print('val_labels  type:', type(val_labels))


    print('train_data length', len(train_data))
    print('train_labels length', len(train_labels))
    print('val_data length:', len(val_data))
    print('val_labels length:', len(val_labels))

    
    # Convert to numpy array since TF likes those better than lists
    train_data = np.asarray(train_data,dtype=np.float64)
    train_labels = np.asarray(train_labels)
    val_data = np.asarray(val_data,dtype=np.float64)
    val_labels = np.asarray(val_labels)

    print('train_data type:',  type(train_data))
    print('train_labels  type:',  type(train_labels))
    print('val_data  type:',  type(val_data))
    print('val_labels  type:', type(val_labels))

    print('train_data.shape:', train_data.shape)
    print('train_labels.shape:', train_labels.shape)
    print('val_data.shape:', val_data.shape)
    print('val_labels.shape:', val_labels.shape)
    
    # reshape train and validation data to 2D for 2D CNN
    dim = train_data.shape
    train_data = train_data.reshape(dim[0],dim[1],dim[2],1)
    dim = val_data.shape
    val_data = val_data.reshape(dim[0],dim[1],dim[2],1)
    
    print('train_data.shape:', train_data.shape)
    print('val_data.shape:', val_data.shape)

    return(train_data,train_labels,val_data,val_labels)

In [7]:
def load_tf(train_data,train_labels,val_data,val_labels):
    train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))
    val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))

    BATCH_SIZE = 64 # Training batch size 
    SHUFFLE_BUFFER_SIZE = 512 # Determines randomness? Need to doublecheck

    # Create training and validation batches

    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    val_dataset = val_dataset.batch(BATCH_SIZE)

    print('train_dataset type: ', type(train_dataset))
    print('val_dataset type: ', type(val_dataset))
    return train_dataset,val_dataset

## Training

In [8]:
def main():
	t1 = time()
	input_directory = '/dataset/HAR/wisdm-dataset/raw/phone/accel' # Dataset on beastie 
	train_data,train_labels,val_data,val_labels = load_data2D(input_directory) 
	train_dataset,val_dataset = load_tf(train_data,train_labels,val_data,val_labels)
	t2 = time()
	print('Loading data in {} seconds.'.format(round(t2-t1)))
	t1 = time()    
	model = cmodel.create_model2D() # Change model type if necessary
	a,l,vl,va = cmodel.train_model(model,train_dataset,val_dataset,model_name="HARv01_00",show_stats = False) # Change name or show plots directly if desired
	t2 = time()
	print('Training DeepCNN in {} seconds.'.format(round(t2-t1)))

if __name__ == '__main__':
	main()

total number of files:  52
Number of files for trainig:  41
Number of files for validation:  5
Number of files for testing:  5


IndexError: too many indices for array